In [2]:
class Film:
    def __init__(self, titel: str, produktionsjahr: int, laenge: int):
        self.titel = titel
        self.produktionsjahr = produktionsjahr
        self.laenge = laenge
        self.vorstellungen = []

    def einfuegen(self, vorstellung):
        self.vorstellungen.append(vorstellung)
        self.vorstellungen.sort(key=lambda v: v.datumUhrzeit)  # Sortieren nach Datum und Uhrzeit

In [3]:
from datetime import datetime

class Kinosaal:
    def __init__(self, nr: int, reihen: int, sitzeProReihe: int):
        self.nr = nr
        self.reihen = reihen
        self.sitzeProReihe = sitzeProReihe
        self.vorstellungen = []

    def einfuegen(self, vorstellung):
        self.vorstellungen.append(vorstellung)

    def istFrei(self, datumUhrzeit: datetime, dauer: int) -> bool:
        for v in self.vorstellungen:
            if abs((v.datumUhrzeit - datumUhrzeit).total_seconds()) < dauer * 60:
                return False
        return True

In [7]:
from datetime import datetime

class Vorstellung:
    BASISPREIS = 10.00
    AUFPREISUEBERLAENGE = 3.50

    def __init__(self, kinosaal, film, datumUhrzeit: datetime):
        self.kinosaal = kinosaal
        self.film = film
        self.datumUhrzeit = datumUhrzeit
        self.sitzbelegung = [[False] * kinosaal.sitzeProReihe for _ in range(kinosaal.reihen)]

    def sucheBesteFreieSitze(self, anzahl: int) -> int:
        for row in range(len(self.sitzbelegung) - 1, -1, -1):  # Von hinten nach vorne suchen
            for col in range(len(self.sitzbelegung[0]) - anzahl + 1):
                if all(not self.sitzbelegung[row][col + i] for i in range(anzahl)):
                    return row * len(self.sitzbelegung[0]) + col + 1
        return 0

    def belegeSitz(self, sitzNr: int) -> bool:
        row, col = divmod(sitzNr - 1, len(self.sitzbelegung[0]))
        if not self.sitzbelegung[row][col]:
            self.sitzbelegung[row][col] = True
            return True
        return False

    def berechneNormalpreis(self) -> float:
        return self.BASISPREIS + (self.AUFPREISUEBERLAENGE if self.film.laenge > 150 else 0)

In [6]:
from datetime import datetime



class KinoManager:
    def __init__(self):
        self.kinosaele = []
        self.filme = []

    def zeigeFilme(self):
        return self.filme

    def zeigeVorstellungenZuFilm(self, film):
        return film.vorstellungen

    def zeigeSitzbelegungZuVorstellung(self, vorstellung):
        return vorstellung.sitzbelegung

    def sucheBesteFreieSitze(self, vorst, anzahl: int) -> int:
        return vorst.sucheBesteFreieSitze(anzahl)

    def belegeSitze(self, vorst, sitznr: int, anzahl: int) -> bool:
        return vorst.belegeSitz(sitznr)

    def erstelleVorstellung(self, film, anzahlPlaetze: int, datumUhrzeit: datetime):
        for saal in self.kinosaele:
            if saal.sitzeProReihe * saal.reihen >= anzahlPlaetze and saal.istFrei(datumUhrzeit, film.laenge):
                vorstellung = Vorstellung(saal, film, datumUhrzeit)
                saal.einfuegen(vorstellung)
                film.einfuegen(vorstellung)
                return vorstellung
        return None


if __name__ == "__main__":
    manager = KinoManager()
    saal1 = Kinosaal(1, 10, 15)
    manager.kinosaele.append(saal1)

    film1 = Film("Inception", 2010, 148)
    manager.filme.append(film1)

    vorstellung1 = manager.erstelleVorstellung(film1, 150, datetime(2025, 3, 10, 20, 0))

    print("Filme im Kino:", [film.titel for film in manager.zeigeFilme()])
    print("Vorstellungen für Inception:", len(manager.zeigeVorstellungenZuFilm(film1)))
    print("Normalpreis für die Vorstellung:", vorstellung1.berechneNormalpreis())

Filme im Kino: ['Inception']
Vorstellungen für Inception: 1
Normalpreis für die Vorstellung: 10.0
